In [1]:
## Importing Qiskit libraries
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, AncillaRegister, Aer, execute, assemble
from qiskit_aer import AerSimulator
from qiskit.circuit import Gate
from qiskit.visualization import plot_histogram, circuit_drawer
import matplotlib.pyplot as plt
from qiskit.quantum_info import Operator
from qiskit.circuit.library import MCMT, RYGate, RXGate, PhaseGate
import numpy as np
np.set_printoptions(threshold=np.inf)
import math
from typing import Union
from numpy import linalg
from scipy.linalg import expm
import pickle
from numba import jit

In [2]:
## Settings
d = 1 # of dimensions
M = 1 # of registers (rough estimate for M is the number of particles in the final state, but M should be set so that p_success can be maximized)
n = 1 # of particles in the initial state
N_abs = 1 # of modes for momenta
m = 1 # mass of particles of projectile


nqubits = N_abs * d + d + 1
N_s = 2 ** (N_abs + 1) # size of the lattice per one dimension
V = N_s ** d # volume
N = N_abs * d + d + 1 # of qubits per one particle (magnitude of momentum + sign + occupation)
s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.

In [3]:
I = np.array([[1, 0], [0, 1]])
H = np.array([[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), -1/np.sqrt(2)]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

In [4]:
#初期状態|0000・・・0>を準備する。
def StateZeros(nqubits):
    State = np.zeros(2**nqubits)
    State[0]=1
    return State

In [5]:
def makeGate(N, nqubit, Gate): # action of Gate to n'th qubit (n = 0...N)
    if nqubit == 0:
        tmp = Gate
    else: tmp = I
    for i in range(N):
        B_tmp = np.zeros((2 ** (i + 2), 2 ** (i + 2)))
        if nqubit == i + 1:
            B_tmp = np.kron(Gate, tmp)
        else: B_tmp = np.kron(I, tmp)
        tmp = B_tmp
    return tmp

# example of use:
# makeGate(N, (N_abs + 1) * d, X)
# makeGate(1, 1, H)

In [6]:
# def transPosition2Momentum(qmom, sign): # if sign is True, we have to transform qmom to qposition
#     if sign == 'position':
#         qmom = qmom - (N / 2)
#     return qmom

In [7]:
# if dagger is True, make an annihilation operator, if False, an creation operator
# ireg means the number of the register acted
# qmom means that the momentum with sign
def annihilationCreationOpPerReg(qmom, ireg, dagger, N, M):
    if qmom > 0:
        qrow = int(qmom - 1/2 + 2 ** (N-1) + 2 ** N_abs)
    elif qmom < 0:
        qrow = int(np.abs(qmom + 1/2) + 2 ** (N-1))
    tmp_op = np.zeros((2 ** N, 2 ** N))
    if dagger == False:
        tmp_op[0][qrow] = 1
    else:
        tmp_op[qrow][0] = 1
    identity = np.eye(2 ** N)
    if ireg == 0:
        tmp = tmp_op
    else: tmp = identity
    # print(tmp)
    for i in range(M-1):
        # print(i)
        B_tmp = np.zeros((2 ** (i + 2), 2 ** (i + 2)))
        if ireg == i + 1:
            B_tmp = np.kron(tmp_op, tmp)
        else:
            B_tmp = np.kron(identity, tmp)
        tmp = B_tmp
    return tmp

def annihilationCreationOpPerRegInPos(ipos, ireg, dagger, N, M):
    qrow = int(ipos + 2 ** (N-1))
    tmp_op = np.zeros((2 ** N, 2 ** N))
    if dagger == False:
        tmp_op[0][qrow] = 1
    else:
        tmp_op[qrow][0] = 1
    identity = np.eye(2 ** N)
    if ireg == 0:
        tmp = tmp_op
    else: tmp = identity
    # print(tmp)
    for i in range(M-1):
        # print(i)
        B_tmp = np.zeros((2 ** (i + 2), 2 ** (i + 2)))
        if ireg == i + 1:
            B_tmp = np.kron(tmp_op, tmp)
        else:
            B_tmp = np.kron(identity, tmp)
        tmp = B_tmp
    return tmp

def annihilationCreationOp(qmom, dagger, sign):
    tmp_op = np.zeros((2 ** (N * M), 2 ** (N * M)))
    # print(tmp_op.size)
    for ireg in range(M):
        # print(annahilationCreationOpPerReg(qmom, ireg, dagger, N, M).size)
        if sign == 'momentum':
            tmp_op += annihilationCreationOpPerReg(qmom, ireg, dagger, N, M)
        elif sign == 'position':
            tmp_op += annihilationCreationOpPerRegInPos(qmom, ireg, dagger, N, M)
        else: print('sign should be momentum or position!')
    return tmp_op * (1/np.sqrt(M))

# example of use
# N_abs = 2
# N = 4
# M = 1
# annahilationCreationOpPerRegInPos(2, 0, False, N, M)

In [8]:
# to ajustment the dimension of Hilbert space with QC algorithnm, we must add ancilla qubits
def addAncilla(mat, N_abs, n, d, s):
    free_num = (N_abs + math.log2(n)) * d
    bose_num = s + 1
    anc_eye_mat = np.eye(2 ** int(free_num + bose_num))
    return np.kron(anc_eye_mat, mat)

In [9]:
import cmath
# flip alignment code and QFT code
def changeBasis(N):
    tmp = np.eye(2 ** N)
    for i in range(2 ** (N - 2)):
        tmp[2 ** (N - 1) + i][2 ** (N - 1) + i] = 0
        tmp[2 ** (N - 1) + i][2 ** (N - 1) + 2 **(N - 2) - i - 1] = 1
    return tmp

def dft_matrix(num):
    A = np.arange(num)
    B = A.reshape(1, -1)
    C = A.reshape(-1, 1)
    M = cmath.e**(-1j * 2 * cmath.pi * B * C  / num)
    return M * (1/np.sqrt(num))

def qft(N):
    return np.kron(np.eye(2), dft_matrix(2 ** (N - 1)))

In [10]:
def makeGatePerReg(Gate, M): # make tensor M products of Gate 
    tmp = Gate
    for i in range(M - 1):
        B_tmp = np.zeros((2 ** (N * i), 2 ** (N * i)))
        B_tmp = np.kron(Gate, tmp)
        tmp = B_tmp
    return tmp

In [11]:
# annahilationCreationOpPerReg(3/2, 0, False, 3, 2, 'momentum')
# annahilationCreationOp(1, True)

In [42]:
def momentum2omega(qmom, m, a_s):
    N_s = 2 ** (N_abs + 1) # number of lattice per dimension
    pmom = ((2 * np.pi) / (N_s * a_s)) * qmom
    omega = np.sqrt(pmom ** 2 + m ** 2)
    return omega

In [44]:
def sqij(qmom, ireg, jreg):
    zq = (1/2) * np.log(np.abs(qmom))
    A = annihilationCreationOpPerReg(qmom, ireg, True, N, M)
    B = annihilationCreationOpPerReg(-qmom, jreg, True, N, M)
    C = annihilationCreationOpPerReg(-qmom, jreg, False, N, M)
    D = annihilationCreationOpPerReg(qmom, ireg, False, N, M)
    ln_sq = -(zq/M) * (A @ B - C @ D)
    sq_ij = expm(ln_sq)
    return sq_ij

def squeezeOpPerMom(qmom, m, a_s):
    omega = momentum2omega(qmom, m, a_s)
    zq = (1/2) * np.log(omega)
    A = annihilationCreationOp(qmom, True, 'momentum')
    # print('A is', A)
    B = annihilationCreationOp(-qmom, True, 'momentum')
    # print('B is', B)
    C = annihilationCreationOp(-qmom, False, 'momentum')
    # print('C is', C)
    D = annihilationCreationOp(qmom, False, 'momentum')
    # print(linalg.norm(A @ B, 2))
    ln_sq = -zq * (A @ B - C @ D)
    sq = expm(ln_sq)
    return sq

def squeezeOp(m, a_s): # corresponds to \prod exp{\sum_{i} a^{i} + a^{i}^{dagger}}
    sq = np.eye(2 ** (N * M))
    for q in range(N_abs + 1):
        # print(q)
        # print(squeezeOpPerMom(q+1/2))
        # print(squeezeOpPerMom(-(q+1/2)))
        sq @= squeezeOpPerMom(q + 1/2, m, a_s)
        # sq @= squeezeOpPerMom(-(q + 1/2))
    return sq

In [72]:
def squeezeOpTrue(m, a_s):
    ln_sq = np.zeros((2 ** (N * M), 2 ** (N * M)))
    for s in range(2):
        sign = (-1) ** s
        for q in range(2 ** (N_abs * d)):
            qmom = (q + 1/2) * sign
            # print('qmom', qmom)
            omega = momentum2omega(qmom, m, a_s)
            zq = (1/2) * np.log(omega)
            A = annihilationCreationOp(qmom, True, 'momentum')
            B = annihilationCreationOp(-qmom, True, 'momentum')
            C = annihilationCreationOp(-qmom, False, 'momentum')
            D = annihilationCreationOp(qmom, False, 'momentum')
            ln_sq += -zq * (A @ B - C @ D)
    return expm(ln_sq)

In [14]:
def phi(n):
    A = annihilationCreationOp(n, True, 'position')
    B = annihilationCreationOp(n, False, 'position')
    phiGate = (1/np.sqrt(2)) * (A + B)
    return phiGate

# exact value of interaction Hamiltonian time evolution
def intU(delta, lam):
    ln_UI = 0
    for ipos in range(2 ** (N_abs * d + d)):
        ln_UI += -1j * ((delta * lam) / 24) * (phi(ipos) @ phi(ipos) @ phi(ipos) @ phi(ipos))
        print(expm(-1j * ((delta * lam) / 24) * (phi(ipos) @ phi(ipos) @ phi(ipos) @ phi(ipos))))
    uI = expm(ln_UI)
    return uI

# approximation for commutation error in [a_m, a_n^dagger]
def intUver2(delta, lam):
    uI = np.eye(2 ** (N * M), dtype='complex128')
    for ipos in range(2 ** (N_abs * d + d)):
        # print(expm(-1j * ((delta * lam) / 24) * (phi(ipos) @ phi(ipos) @ phi(ipos) @ phi(ipos))))
        uI @= expm(-1j * ((delta * lam) / 24) * (phi(ipos) @ phi(ipos) @ phi(ipos) @ phi(ipos)))
    return uI

In [15]:
def commutation(phi1, phi2):
    return phi1 @ phi2 -phi2 @ phi1

In [16]:
# This code shows each phi(i.e. a_n and a_n'^dagger)deos not commute
# M = 5
# count = 0
# comm = commutation(phi(1), phi(2))
# for i in range(phi(1)[0].size):
#     for j in range(phi(1)[0].size):
#         if comm[i][j] != 0:
#             print(comm[i][j])
#             # print(comm[i][j])
#             count += 1
#         if count == 1:
#             break
#     if count == 1:
#         break
# print(count)

In [17]:
def threeIndicesOp(ireg, jreg, ipos, Delta):
    A = annihilationCreationOpPerRegInPos(ipos, ireg, False, N, M) + annihilationCreationOpPerRegInPos(ipos, ireg, True, N, M)
    B = annihilationCreationOpPerRegInPos(ipos, jreg, False, N, M) + annihilationCreationOpPerRegInPos(ipos, jreg, True, N, M)
    ln_exp = -1j * Delta * A @ B
    return expm(ln_exp)

def allDifferOp(ireg, jreg, kreg, lreg, ipos, Delta):
    A = annihilationCreationOpPerRegInPos(ipos, ireg, False, N, M) + annihilationCreationOpPerRegInPos(ipos, ireg, True, N, M)
    B = annihilationCreationOpPerRegInPos(ipos, jreg, False, N, M) + annihilationCreationOpPerRegInPos(ipos, jreg, True, N, M)
    C = annihilationCreationOpPerRegInPos(ipos, kreg, False, N, M) + annihilationCreationOpPerRegInPos(ipos, kreg, True, N, M)
    D = annihilationCreationOpPerRegInPos(ipos, lreg, False, N, M) + annihilationCreationOpPerRegInPos(ipos, lreg, True, N, M)
    ln_exp = -1j * Delta * (A @ B @ C @ D)
    return expm(ln_exp)

In [18]:
# M = 2
# N_abs = 1
# d = 1
# N = N_abs * d + d + 1
# n = 1

# delta = 1
# lam = 20
# Delta = (delta * lam) / (96 * (M ** 2))
# # mat = addAncilla(threeIndicesOp(ireg=0, jreg=1, ipos=1, Delta=Delta), N_abs, n, d, s)
# mat = addAncilla(allDifferOp(ireg=0, jreg=0, kreg=1, lreg=1, ipos=2, Delta=Delta), N_abs, n, d, s)

In [74]:
# M = 3
# mat = np.eye(2 ** (N * M), dtype='complex128')
# mat = addAncilla(mat, N_abs, n, d, s)
# # for ipos in range(2 ** ((N_abs * d + d))):
# # 2 indices match
# for ireg in range(M):
#     for jreg in range(M):
#         for kreg in range(M):
#             if ireg != jreg and ireg != kreg and jreg < kreg:
#                 lreg = ireg
#                 print('ireg, jreg, kreg', ireg, jreg, kreg)
#                 mat @= addAncilla(allDifferOp(ireg, jreg, kreg, lreg, ipos=2, Delta=Delta), N_abs, n, d, s)

In [19]:
# M = 2
# tmp = np.eye(2 ** (N * M), dtype='complex128')
# for ipos in range(2 ** (N_abs * d + d)):
# #     dummy = 0
# # if dummy == 0:
#     for ireg in range(M):
#         for jreg in range(M):
#             for kreg in range(M):
#                 for lreg in range(M):
#                     print('i, j, k, l', ireg, jreg, kreg, lreg)
#                     tmp @= allDifferOp(ireg, jreg, kreg, lreg, ipos, Delta=Delta)
#     # print('ipos', ipos)
# mat = tmp

In [20]:
# len(mat)
# list = []
# for i in range(len(mat[0])):
#     if mat[i][0] != 0:
#         list.append(mat[i][0])
#         print(i)
# list

In [21]:
# iposition = 1
# M = 2
# N_abs = 1
# d = 1
# N = N_abs * d + d + 1
# n = 1
# delta = 1.5
# coupling_lambda = 1
# Delta = (10 * delta * coupling_lambda) / (96 * (M ** 2))

# tmp = np.eye(2 ** (N * M), dtype='complex128')
# for ireg in range(M):
#     for jreg in range(M):
#         for kreg in range(M):
#             for lreg in range(M):
#                 tmp @= allDifferOp(ireg, jreg, kreg, lreg, iposition, Delta)
# sigma_value = tmp

# # phi_value = expm(-1j * Delta * 4 * (M ** 2) * phi(1) @ phi(1) @ phi(1) @ phi(1))

# sigma_value

# len(sigma_value)
# list = []
# for i in range(len(sigma_value[0])):
#     if sigma_value[i][0] != 0:
#         list.append(sigma_value[i][0])
#         print(i)
# list

In [22]:
# M = 2
# N_abs = 1
# d = 1
# N = N_abs * d + d + 1
# n = 1
# delta = 1.5
# coupling_lambda = 100
# lambda_t = coupling_lambda
# Delta = (delta * coupling_lambda) / (96 * (M ** 2))
# free_num = int((N_abs + math.log2(n)) * d)
# bose_num = int(s + 1)
# state = StateZeros(N * M)

# tmp = np.eye(2 ** (N * M), dtype='complex128')
# for iposition in range(2 ** (N_abs * d + d)):
#     for ireg in range(M):
#         for jreg in range(M):
#             for kreg in range(M):
#                 for lreg in range(M):
#                     tmp @= allDifferOp(ireg, jreg, kreg, lreg, iposition, Delta)

# U_I = intU(delta, lambda_t)

# ket = np.dot(U_I, state)
# bra = np.dot(tmp, state)
# bra = np.conjugate(bra).T

# np.abs(np.dot(bra, ket))

In [71]:
M = 3
N_abs = 1
d = 1
N = N_abs * d + d + 1
n = 1
s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.
tau_0 = 0.5 # Wigner delay
tau_I = 0 # length of time of interaction
delta = 1 # time step
m_0 = 0 # chosen to represent a relevant enrgy scale in the weak coupling regime
tau = 0
m_t = m_0
lambda_t = 1
a_s = 1

m_ren = m_0
lambda_ren = 1
free_num = int((N_abs + math.log2(n)) * d)
bose_num = int(s + 1)
state = StateZeros(((N * M) + (free_num + bose_num)))
count = 0

while tau <= tau_0 + tau_I:
    print('count', count)
    # Squeezing operation
    # S = addAncilla(squeezeOp(), N_abs, n, d, s)
    S = addAncilla(squeezeOpTrue(m, a_s), N_abs, n, d, s)
    state = np.dot(S, state)

    # # Change basis
    # C = addAncilla(makeGatePerReg(changeBasis(N), M), N_abs, n, d, s)
    # state = np.dot(C, state)

    # # QFT
    # Q = addAncilla(makeGatePerReg(qft(N), M), N_abs, n, d, s)
    # state = np.dot(Q, state)

    # # # Interaction Hamiltonian
    # # print(lambda_t)
    # U_I = addAncilla(intU(delta, lambda_t), N_abs, n, d, s)
    # # U_I = addAncilla(intUver2(delta, lambda_t), N_abs, n, d, s)
    # state = np.dot(U_I, state)

    # # inverse QFT
    # Q = addAncilla(makeGatePerReg(qft(N), M), N_abs, n, d, s)
    # invQ = np.linalg.inv(Q)
    # state = np.dot(invQ, state)

    # # Change basis
    # C = addAncilla(makeGatePerReg(changeBasis(N), M), N_abs, n, d, s)
    # state = np.dot(C, state)
    count += 1

    tau += delta
    if tau < tau_0: # turn on interaction adiabatically
        lambda_t = lambda_ren * (tau / tau_0)
        m_t = m_0 * (1 - (tau / tau_0)) + m_ren * (tau / tau_0)
    elif tau_0 <= tau : # interaction completely
        m_t = m_ren
        lambda_t = lambda_ren

print(state)

# 結果を保存
result_file = 'calculated_result.pkl'
with open(result_file, 'wb') as f:
    pickle.dump(state, f)

count 0
qmom 0.5
qmom 1.5
qmom -0.5
qmom -1.5
[ 0.70237721  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.07196551  0.          0.          0.
  0.          0.          0.          0.          0.         -0.28154101
  0.          0.          0.          0.         -0.07196551  0.
  0.          0.          0.          0.          0.          0.
  0.         -0.28154101  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.        

In [51]:
list = []
for i in range(len(state)):
    if state[i] != 0:
        list.append(state[i])
        print(i)
list

0
38
47
52
61


[0.9417586936101052,
 -0.05888941566524289,
 -0.23038515192498893,
 -0.058889415665242896,
 -0.23038515192498896]